## **Applio NoUI**
A simple, high-quality voice conversion tool focused on ease of use and performance.

[Support](https://discord.gg/urxFjYmYYh) — [GitHub](https://github.com/IAHispano/Applio)

<br>

### **Credits**
- Encryption method: [Hina](https://github.com/hinabl)
- Extra section: [Poopmaster](https://github.com/poiqazwsx)
- Main development: [Applio Team](https://github.com/IAHispano)
- Colab inspired on [RVC v2 Disconnected](https://colab.research.google.com/drive/1XIPCP9ken63S7M6b5ui1b36Cs17sP-NS).

### If you restart the runtime, run it again.

In [ ]:
%cd /content/Applio

# Installation
## If the runtime restarts, run the cell above and re-run the installation steps.

In [ ]:
# @title Mount Google Drive
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
# @title Clone
!git clone https://github.com/IAHispano/Applio --branch 3.2.8-bugfix --single-branch
%cd /content/Applio

In [ ]:
# @title Install
rot_47 = lambda encoded_text: "".join(
    [
        (
            chr(
                (ord(c) - (ord("a") if c.islower() else ord("A")) - 47) % 26
                + (ord("a") if c.islower() else ord("A"))
            )
            if c.isalpha()
            else c
        )
        for c in encoded_text
    ]
)
import codecs
import os
import tarfile
import subprocess
from pathlib import Path
from IPython.display import clear_output

def vidal_setup(C):
    def F():
        print("Installing pip packages...")
        subprocess.check_call(["pip", "install", "-r", "requirements.txt", "--quiet"])

    A = "/content/" + rot_47("Kikpm.ovm.bu")
    D = "/"
    if not os.path.exists(A):
        M = os.path.dirname(A)
        os.makedirs(M, exist_ok=True)
        print("No cached install found..")
        try:
            N = codecs.decode(
                    "uggcf://uhttvatsnpr.pb/VNUvfcnab/Nccyvb/erfbyir/znva/Raivebzrag/Pbyno/Cache.gne.tm",
                    "rot_13",
                )
            subprocess.run(["wget", "-O", A, N])
            print("Download completed successfully!")
        except Exception as H:
            print(str(H))
            if os.path.exists(A):
                os.remove(A)
    if Path(A).exists():
        with tarfile.open(A, "r:gz") as I:
            I.extractall(D)
            print(f"Extraction of {A} to {D} completed.")
        if os.path.exists(A):
            os.remove(A)
        if C:
            F()
            C = False
    else:
        F()


vidal_setup(False)
!pip uninstall torch torchvision torchaudio -y
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --upgrade --index-url https://download.pytorch.org/whl/cu121
clear_output()
print("Finished installing requirements!")

In [ ]:
# @title Download models
!python core.py "prerequisites" --models "True" --exe "True" --pretraineds_v1_f0 "False" --pretraineds_v2_f0 "True" --pretraineds_v1_nof0 "False" --pretraineds_v2_nof0 "False"

# Infer


In [ ]:
# @title Download model
# @markdown Hugging Face or Google Drive
model_link = "https://huggingface.co/Darwin/Darwin/resolve/main/Darwin.zip"  # @param {type:"string"}

!python core.py download --model_link "{model_link}"

In [ ]:
# @title Run Inference
# @markdown Please upload the audio file to your Google Drive path `/content/drive/MyDrive` and specify its name here. For the model name, use the zip file name without the extension. Alternatively, you can check the path `/content/Applio/logs` for the model name (name of the folder).

import os

current_dir = os.getcwd()

model_name = "Darwin"  # @param {type:"string"}
model_folder = os.path.join(current_dir, f"logs/{model_name}")

if not os.path.exists(model_folder):
    raise FileNotFoundError(f"Model directory not found: {model_folder}")

files_in_folder = os.listdir(model_folder)
pth_path = next((f for f in files_in_folder if f.endswith(".pth")), None)
index_file = next((f for f in files_in_folder if f.endswith(".index")), None)

if pth_path is None or index_file is None:
    raise FileNotFoundError("No model found.")

pth_file = os.path.join(model_folder, pth_path)
index_file = os.path.join(model_folder, index_file)

input_path = "/content/example.wav"  # @param {type:"string"}
output_path = "/content/output.wav"
export_format = "WAV"  # @param ['WAV', 'MP3', 'FLAC', 'OGG', 'M4A'] {allow-input: false}
f0_method = "rmvpe"  # @param ["crepe", "crepe-tiny", "rmvpe", "fcpe", "hybrid[rmvpe+fcpe]"] {allow-input: false}
f0_up_key = 0  # @param {type:"slider", min:-24, max:24, step:0}
filter_radius = 3  # @param {type:"slider", min:0, max:10, step:0}
rms_mix_rate = 0.8  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
protect = 0.5  # @param {type:"slider", min:0.0, max:0.5, step:0.1}
index_rate = 0.7  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
hop_length = 128  # @param {type:"slider", min:1, max:512, step:0}
clean_strength = 0.7  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
split_audio = False  # @param{type:"boolean"}
clean_audio = False  # @param{type:"boolean"}
f0_autotune = False  # @param{type:"boolean"}
formant_shift = False # @param{type:"boolean"}
formant_qfrency = 1.0 # @param {type:"slider", min:1.0, max:16.0, step:0.1}
formant_timbre = 1.0 # @param {type:"slider", min:1.0, max:16.0, step:0.1}
embedder_model = "contentvec" # @param ["contentvec", "chinese-hubert-base", "japanese-hubert-base", "korean-hubert-base", "custom"] {allow-input: false}
embedder_model_custom = "" # @param {type:"string"}


# Post-processing effects
if "post_process" not in globals():
  post_process = False
if "reverb" not in globals():
  reverb = False
if "pitch_shift" not in globals():
  pitch_shift = False
if "limiter" not in globals():
  limiter = False
if "gain" not in globals():
  gain = False
if "distortion" not in globals():
  distortion = False
if "chorus" not in globals():
  chorus = False
if "bitcrush" not in globals():
  bitcrush = False
if "clipping" not in globals():
  clipping = False
if "compressor" not in globals():
  compressor = False
if "delay" not in globals():
  delay = False

if "reverb_room_size" not in globals():
  reverb_room_size = 0.5
if "reverb_damping" not in globals():
  reverb_damping = 0.5
if "reverb_wet_gain" not in globals():
  reverb_wet_gain = 0.0
if "reverb_dry_gain" not in globals():
  reverb_dry_gain = 0.0
if "reverb_width" not in globals():
  reverb_width = 1.0
if "reverb_freeze_mode" not in globals():
  reverb_freeze_mode = 0.0

if "pitch_shift_semitones" not in globals():
  pitch_shift_semitones = 0.0

if "limiter_threshold" not in globals():
  limiter_threshold = -1.0
if "limiter_release_time" not in globals():
  limiter_release_time = 0.05

if "gain_db" not in globals():
  gain_db = 0.0

if "distortion_gain" not in globals():
  distortion_gain = 0.0

if "chorus_rate" not in globals():
  chorus_rate = 1.5
if "chorus_depth" not in globals():
  chorus_depth = 0.1
if "chorus_center_delay" not in globals():
  chorus_center_delay = 15.0
if "chorus_feedback" not in globals():
  chorus_feedback = 0.25
if "chorus_mix" not in globals():
  chorus_mix = 0.5

if "bitcrush_bit_depth" not in globals():
  bitcrush_bit_depth = 4

if "clipping_threshold" not in globals():
  clipping_threshold = 0.5

if "compressor_threshold" not in globals():
  compressor_threshold = -20.0
if "compressor_ratio" not in globals():
  compressor_ratio = 4.0
if "compressor_attack" not in globals():
  compressor_attack = 0.001
if "compressor_release" not in globals():
  compressor_release = 0.1

if "delay_seconds" not in globals():
  delay_seconds = 0.1
if "delay_feedback" not in globals():
  delay_feedback = 0.5
if "delay_mix" not in globals():
  delay_mix = 0.5

!python core.py infer --pitch "{f0_up_key}" --filter_radius "{filter_radius}" --volume_envelope "{rms_mix_rate}" --index_rate "{index_rate}" --hop_length "{hop_length}" --protect "{protect}" --f0_autotune "{f0_autotune}" --f0_method "{f0_method}" --input_path "{input_path}" --output_path "{output_path}" --pth_path "{pth_file}" --index_path "{index_file}" --split_audio "{split_audio}" --clean_audio "{clean_audio}" --clean_strength "{clean_strength}" --export_format "{export_format}" --embedder_model "{embedder_model}" --embedder_model_custom "{embedder_model_custom}" --formant_shifting "{formant_shift}" --formant_qfrency "{formant_qfrency}" --formant_timbre "{formant_timbre}" --post_process "{post_process}" --reverb "{reverb}" --pitch_shift "{pitch_shift}" --limiter "{limiter}" --gain "{gain}" --distortion "{distortion}" --chorus "{chorus}" --bitcrush "{bitcrush}" --clipping "{clipping}" --compressor "{compressor}" --delay "{delay}" --reverb_room_size "{reverb_room_size}" --reverb_damping "{reverb_damping}" --reverb_wet_gain "{reverb_wet_gain}" --reverb_dry_gain "{reverb_dry_gain}" --reverb_width "{reverb_width}" --reverb_freeze_mode "{reverb_freeze_mode}" --pitch_shift_semitones "{pitch_shift_semitones}" --limiter_threshold "{limiter_threshold}" --limiter_release_time "{limiter_release_time}" --gain_db "{gain_db}" --distortion_gain "{distortion_gain}" --chorus_rate "{chorus_rate}" --chorus_depth "{chorus_depth}" --chorus_center_delay "{chorus_center_delay}" --chorus_feedback "{chorus_feedback}" --chorus_mix "{chorus_mix}" --bitcrush_bit_depth "{bitcrush_bit_depth}" --clipping_threshold "{clipping_threshold}" --compressor_threshold "{compressor_threshold}" --compressor_ratio "{compressor_ratio}" --compressor_attack "{compressor_attack}" --compressor_release "{compressor_release}" --delay_seconds "{delay_seconds}" --delay_feedback "{delay_feedback}" --delay_mix "{delay_mix}"

from IPython.display import Audio, display, clear_output

output_path = output_path.replace(".wav", f".{export_format.lower()}")
# clear_output()
display(Audio(output_path, autoplay=True))

## **Advanced Settings**

In [ ]:
# @title # Post-processing effects
post_process = False # @param{type:"boolean"}
reverb = False # @param{type:"boolean"}
pitch_shift = False # @param{type:"boolean"}
limiter = False # @param{type:"boolean"}
gain = False # @param{type:"boolean"}
distortion = False # @param{type:"boolean"}
chorus = False # @param{type:"boolean"}
bitcrush = False # @param{type:"boolean"}
clipping = False # @param{type:"boolean"}
compressor = False # @param{type:"boolean"}
delay = False # @param{type:"boolean"}

reverb_room_size = 0.5 # @param {type:"slider", min:0.0, max:1.0, step:0.1}
reverb_damping = 0.5 # @param {type:"slider", min:0.0, max:1.0, step:0.1}
reverb_wet_gain = 0.0 # @param {type:"slider", min:-20.0, max:20.0, step:0.1}
reverb_dry_gain = 0.0 # @param {type:"slider", min:-20.0, max:20.0, step:0.1}
reverb_width = 1.0 # @param {type:"slider", min:0.0, max:1.0, step:0.1}
reverb_freeze_mode = 0.0 # @param {type:"slider", min:0.0, max:1.0, step:0.1}

pitch_shift_semitones = 0.0 # @param {type:"slider", min:-12.0, max:12.0, step:0.1}

limiter_threshold = -1.0 # @param {type:"slider", min:-20.0, max:0.0, step:0.1}
limiter_release_time = 0.05 # @param {type:"slider", min:0.0, max:1.0, step:0.01}

gain_db = 0.0 # @param {type:"slider", min:-20.0, max:20.0, step:0.1}

distortion_gain = 0.0 # @param {type:"slider", min:0.0, max:1.0, step:0.1}

chorus_rate = 1.5 # @param {type:"slider", min:0.1, max:10.0, step:0.1}
chorus_depth = 0.1 # @param {type:"slider", min:0.0, max:1.0, step:0.1}
chorus_center_delay = 15.0 # @param {type:"slider", min:0.0, max:50.0, step:0.1}
chorus_feedback = 0.25 # @param {type:"slider", min:0.0, max:1.0, step:0.1}
chorus_mix = 0.5 # @param {type:"slider", min:0.0, max:1.0, step:0.1}

bitcrush_bit_depth = 4 # @param {type:"slider", min:1, max:16, step:1}

clipping_threshold = 0.5 # @param {type:"slider", min:0.0, max:1.0, step:0.1}

compressor_threshold = -20.0 # @param {type:"slider", min:-60.0, max:0.0, step:0.1}
compressor_ratio = 4.0 # @param {type:"slider", min:1.0, max:20.0, step:0.1}
compressor_attack = 0.001 # @param {type:"slider", min:0.0, max:0.1, step:0.001}
compressor_release = 0.1 # @param {type:"slider", min:0.0, max:1.0, step:0.01}

delay_seconds = 0.1 # @param {type:"slider", min:0.0, max:1.0, step:0.01}
delay_feedback = 0.5 # @param {type:"slider", min:0.0, max:1.0, step:0.1}
delay_mix = 0.5 # @param {type:"slider", min:0.0, max:1.0, step:0.1}


# Train

In [ ]:
# @title Preprocess Dataset
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
model_name = "Darwin"  # @param {type:"string"}
dataset_path = "/content/drive/MyDrive/Darwin_Dataset"  # @param {type:"string"}

sample_rate = "40k"  # @param ["32k", "40k", "48k"] {allow-input: false}
sr = int(sample_rate.rstrip("k")) * 1000
cpu_cores = 2 # @param {type:"slider", min:1, max:2, step:1}
cut_preprocess = True # @param{type:"boolean"}
process_effects = False # @param{type:"boolean"}
noise_reduction = False # @param{type:"boolean"}
noise_reduction_strength = 0.7 # @param {type:"slider", min:0.0, max:1.0, step:0.1}

!python core.py preprocess --model_name "{model_name}" --dataset_path "{dataset_path}" --sample_rate "{sr}" --cpu_cores "{cpu_cores}" --cut_preprocess "{cut_preprocess}" --process_effects "{process_effects}" --noise_reduction "{noise_reduction}" --noise_reduction_strength "{noise_reduction_strength}"

In [ ]:
# @title Extract Features
f0_method = "rmvpe"  # @param ["crepe", "crepe-tiny", "rmvpe"] {allow-input: false}
hop_length = 128  # @param {type:"slider", min:1, max:512, step:0}

sr = int(sample_rate.rstrip("k")) * 1000
cpu_cores = 2 # @param {type:"slider", min:1, max:2, step:1}
embedder_model = "contentvec" # @param ["contentvec", "chinese-hubert-base", "japanese-hubert-base", "korean-hubert-base", "custom"] {allow-input: false}
embedder_model_custom = "" # @param {type:"string"}

!python core.py extract --model_name "{model_name}" --f0_method "{f0_method}" --hop_length "{hop_length}" --sample_rate "{sr}" --cpu_cores "{cpu_cores}" --gpu "0" --embedder_model "{embedder_model}" --embedder_model_custom "{embedder_model_custom}"

In [ ]:
# @title Train
import threading
import time
import os
import shutil
import hashlib
import time

LOGS_FOLDER = "/content/Applio/logs/"
GOOGLE_DRIVE_PATH = "/content/drive/MyDrive/RVC_Backup"


def import_google_drive_backup():
    print("Importing Google Drive backup...")
    for root, dirs, files in os.walk(GOOGLE_DRIVE_PATH):
        for filename in files:
            filepath = os.path.join(root, filename)
            if os.path.isfile(filepath):
                backup_filepath = os.path.join(
                    LOGS_FOLDER, os.path.relpath(filepath, GOOGLE_DRIVE_PATH)
                )
                backup_folderpath = os.path.dirname(backup_filepath)
                if not os.path.exists(backup_folderpath):
                    os.makedirs(backup_folderpath)
                    print(f"Created backup folder: {backup_folderpath}", flush=True)
                shutil.copy2(filepath, backup_filepath)
                print(f"Imported file from Google Drive backup: {filename}")
    print("Google Drive backup import completed.")


def get_md5_hash(file_path):
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()


if "autobackups" not in globals():
    autobackups = False
# @markdown ### 💾 AutoBackup
cooldown = 15  # @param {type:"slider", min:0, max:100, step:0}
auto_backups = True  # @param{type:"boolean"}
def backup_files():
    print("\nStarting backup loop...")
    last_backup_timestamps_path = os.path.join(
        LOGS_FOLDER, "last_backup_timestamps.txt"
    )
    fully_updated = False

    while True:
        try:
            updated_files = 0
            deleted_files = 0
            new_files = 0
            last_backup_timestamps = {}

            try:
                with open(last_backup_timestamps_path, "r") as f:
                    last_backup_timestamps = dict(line.strip().split(":") for line in f)
            except FileNotFoundError:
                pass

            for root, dirs, files in os.walk(LOGS_FOLDER):
                # Excluding "zips" and "mute" directories
                if "zips" in dirs:
                    dirs.remove("zips")
                if "mute" in dirs:
                    dirs.remove("mute")

                for filename in files:
                    if filename != "last_backup_timestamps.txt":
                        filepath = os.path.join(root, filename)
                        if os.path.isfile(filepath):
                            backup_filepath = os.path.join(
                                GOOGLE_DRIVE_PATH,
                                os.path.relpath(filepath, LOGS_FOLDER),
                            )
                            backup_folderpath = os.path.dirname(backup_filepath)
                            if not os.path.exists(backup_folderpath):
                                os.makedirs(backup_folderpath)
                            last_backup_timestamp = last_backup_timestamps.get(filepath)
                            current_timestamp = os.path.getmtime(filepath)
                            if (
                                last_backup_timestamp is None
                                or float(last_backup_timestamp) < current_timestamp
                            ):
                                shutil.copy2(filepath, backup_filepath)
                                last_backup_timestamps[filepath] = str(current_timestamp)
                                if last_backup_timestamp is None:
                                    new_files += 1
                                else:
                                    updated_files += 1


            for filepath in list(last_backup_timestamps.keys()):
                if not os.path.exists(filepath):
                    backup_filepath = os.path.join(
                        GOOGLE_DRIVE_PATH, os.path.relpath(filepath, LOGS_FOLDER)
                    )
                    if os.path.exists(backup_filepath):
                        os.remove(backup_filepath)
                        deleted_files += 1
                    del last_backup_timestamps[filepath]


            if updated_files > 0 or deleted_files > 0 or new_files > 0:
                print(f"Backup Complete: {new_files} new, {updated_files} updated, {deleted_files} deleted.")
                fully_updated = False
            elif not fully_updated:
                print("Files are up to date.")
                fully_updated = True

            with open(last_backup_timestamps_path, "w") as f:
                for filepath, timestamp in last_backup_timestamps.items():
                    f.write(f"{filepath}:{timestamp}\n")

            time.sleep(cooldown if fully_updated else 0.1)


        except Exception as error:
            print(f"An error occurred during backup: {error}")


if autobackups:
    autobackups = False
    print("Autobackup Disabled")
else:
    autobackups = True
    print("Autobackup Enabled")
# @markdown ### ⚙️ Train Settings
total_epoch = 800  # @param {type:"integer"}
batch_size = 15  # @param {type:"slider", min:1, max:25, step:0}
gpu = 0
sr = int(sample_rate.rstrip("k")) * 1000
pretrained = True  # @param{type:"boolean"}
cleanup = False  # @param{type:"boolean"}
cache_data_in_gpu = False  # @param{type:"boolean"}
tensorboard = True  # @param{type:"boolean"}
# @markdown ### ➡️ Choose how many epochs your model will be stored
save_every_epoch = 10  # @param {type:"slider", min:1, max:100, step:0}
save_only_latest = False  # @param{type:"boolean"}
save_every_weights = False  # @param{type:"boolean"}
overtraining_detector = False  # @param{type:"boolean"}
overtraining_threshold = 50  # @param {type:"slider", min:1, max:100, step:0}
# @markdown ### ❓ Optional
# @markdown In case you select custom pretrained, you will have to download the pretraineds and enter the path of the pretraineds.
custom_pretrained = False  # @param{type:"boolean"}
g_pretrained_path = "/content/Applio/rvc/models/pretraineds/pretraineds_custom/G48k.pth"  # @param {type:"string"}
d_pretrained_path = "/content/Applio/rvc/models/pretraineds/pretraineds_custom/D48k.pth"  # @param {type:"string"}

if "pretrained" not in globals():
    pretrained = True

if "custom_pretrained" not in globals():
    custom_pretrained = False

if "g_pretrained_path" not in globals():
    g_pretrained_path = "Custom Path"

if "d_pretrained_path" not in globals():
    d_pretrained_path = "Custom Path"


def start_train():
    if tensorboard == True:
        %load_ext tensorboard
        %tensorboard --logdir /content/Applio/logs/
    !python core.py train --model_name "{model_name}" --save_every_epoch "{save_every_epoch}" --save_only_latest "{save_only_latest}" --save_every_weights "{save_every_weights}" --total_epoch "{total_epoch}" --sample_rate "{sr}" --batch_size "{batch_size}" --gpu "{gpu}" --pretrained "{pretrained}" --custom_pretrained "{custom_pretrained}" --g_pretrained_path "{g_pretrained_path}" --d_pretrained_path "{d_pretrained_path}" --overtraining_detector "{overtraining_detector}" --overtraining_threshold "{overtraining_threshold}" --cleanup "{cleanup}" --cache_data_in_gpu "{cache_data_in_gpu}"


server_thread = threading.Thread(target=start_train)
server_thread.start()

if auto_backups:
    backup_files()
else:
    while True:
        time.sleep(10)

In [ ]:
# @title Generate index file
index_algorithm = "Auto"  # @param ["Auto", "Faiss", "KMeans"] {allow-input: false}
!python core.py index --model_name "{model_name}" --index_algorithm "{index_algorithm}"

In [ ]:
# @title Save model
# @markdown Enter the name of the model and the steps. You can find it in your `/content/Applio/logs` folder.
%cd /content
import os, shutil, sys

model_name = "Darwin"  # @param {type:"string"}
model_epoch = 800  # @param {type:"integer"}
save_big_file = False  # @param {type:"boolean"}

if os.path.exists("/content/zips"):
    shutil.rmtree("/content/zips")
print("Removed zips.")

os.makedirs(f"/content/zips/{model_name}/", exist_ok=True)
print("Created zips.")

logs_folder = f"/content/Applio/logs/{model_name}/"
weight_file = None
if not os.path.exists(logs_folder):
    print(f"Model folder not found.")
    sys.exit("")

for filename in os.listdir(logs_folder):
    if filename.startswith(f"{model_name}_{model_epoch}e") and filename.endswith(".pth"):
        weight_file = filename
        break
if weight_file is None:
    print("There is no weight file with that name")
    sys.exit("")
if not save_big_file:
    !cp {logs_folder}added_*.index /content/zips/{model_name}/
    !cp {logs_folder}total_*.npy /content/zips/{model_name}/
    !cp {logs_folder}{weight_file} /content/zips/{model_name}/
    %cd /content/zips
    !zip -r {model_name}.zip {model_name}
if save_big_file:
    %cd /content/Applio
    latest_steps = -1
    logs_folder = "./logs/" + model_name
    for filename in os.listdir(logs_folder):
        if filename.startswith("G_") and filename.endswith(".pth"):
            steps = int(filename.split("_")[1].split(".")[0])
            if steps > latest_steps:
                latest_steps = steps
    MODELZIP = model_name + ".zip"
    !mkdir -p /content/zips
    ZIPFILEPATH = os.path.join("/content/zips", MODELZIP)
    for filename in os.listdir(logs_folder):
        if "G_" in filename or "D_" in filename:
            if str(latest_steps) in filename:
                !zip -r {ZIPFILEPATH} {os.path.join(logs_folder, filename)}
        else:
            !zip -r {ZIPFILEPATH} {os.path.join(logs_folder, filename)}

!mkdir -p /content/drive/MyDrive/RVC_Backup/
shutil.move(
    f"/content/zips/{model_name}.zip",
    f"/content/drive/MyDrive/RVC_Backup/{model_name}.zip",
)
%cd /content/Applio
shutil.rmtree("/content/zips")

# Resume-training

In [ ]:
# @title Load a Backup
from google.colab import drive
import os
import shutil

# @markdown Put the exact name you put as your Model Name in Applio.
modelname = "My-Project"  # @param {type:"string"}
source_path = "/content/drive/MyDrive/RVC_Backup/" + modelname
destination_path = "/content/Applio/logs/" + modelname
backup_timestamps_file = "last_backup_timestamps.txt"
if not os.path.exists(source_path):
    print(
        "The model folder does not exist. Please verify the name is correct or check your Google Drive."
    )
else:
    time_ = os.path.join("/content/drive/MyDrive/RVC_Backup/", backup_timestamps_file)
    time__ = os.path.join("/content/Applio/logs/", backup_timestamps_file)
    if os.path.exists(time_):
        shutil.copy(time_, time__)
    shutil.copytree(source_path, destination_path)
    print("Model backup loaded successfully.")

In [ ]:
# @title Set training variables
# @markdown ### ➡️ Use the same as you did previously
model_name = "Darwin"  # @param {type:"string"}
sample_rate = "40k"  # @param ["32k", "40k", "48k"] {allow-input: false}
f0_method = "rmvpe"  # @param ["crepe", "crepe-tiny", "rmvpe"] {allow-input: false}
hop_length = 128  # @param {type:"slider", min:1, max:512, step:0}
sr = int(sample_rate.rstrip("k")) * 1000